In [24]:
import genanki
import csv
import argparse
import os
import hashlib
import time
from pathlib import Path

from src.anki import read_file, make_deck, make_model, make_note, make_package, read_package
from src.media import add_images, add_phonetics


INPUT_DIR = Path("input/")
OUTPUT_DIR = Path("output/")
TEMPLATE_DIR = Path("template/")
MEDIA_DIR = Path("media/")
IMG_DIR = MEDIA_DIR / "img"
IMG_DIR.mkdir(parents=True, exist_ok=True)
AUDIO_DIR = MEDIA_DIR / "sound"
AUDIO_DIR.mkdir(parents=True, exist_ok=True)

df_file = INPUT_DIR / "Cours_01.md"
df = read_file(df_file)

deck_name = df_file.name.rsplit(".", maxsplit=1)[-2]
deck_path = OUTPUT_DIR / f"{deck_name}.apkg"

In [25]:
# import logging
# logging.basicConfig(level=logging.INFO, force=True)
import logging
from src.log_config import configure_logging
configure_logging(ignore_libs=["PIL", "phonemizer"])

In [27]:
df = add_phonetics(df)
df, media_files = add_images(df, IMG_DIR)
df.head()

[15:37:48] [INFO] [media] - Added phonetics for 30 words.
[15:37:48] [INFO] [media] - Added 30 images out of 30 words.


,Front,Back,Extra,More,Phonetics,Image,Audio
0,la fenêtre,das Fenster,,,/la- fənˈɛtʁ/,"<img src=""img_83d2b53e093c284e.png"">",
1,Monsieur Macron,Herr Macron,(M. Macron),,/məsjˈø makʁˈɔ̃/,"<img src=""img_c62070d054b986e3.png"">",
2,Madame Borne,Frau Borne,(MMe Borne),,/madˈam bˈɔʁn/,"<img src=""img_b2f6efc7954ab4d9.png"">",
3,Mademoiselle Royal,Fräulein Royal,"(Mlle), veraltet",,/madmwazˈɛl ʁwajˈal/,"<img src=""img_f1d7641526353934.png"">",
4,l'amour,die Liebe,,,/lamˈuʁ/,"<img src=""img_93139178e53667ab.png"">",


In [28]:
style = TEMPLATE_DIR / "style.css"

deck = make_deck(deck_name)
model = make_model(deck_name, df.columns, style)
successful_cards, total_rows = make_note(deck, model, df)
make_package(deck, media_files=media_files, output_dir=OUTPUT_DIR)

logging.info(f"Created {successful_cards} cards out of {total_rows} possible lines.")

[15:37:48] [INFO] [root] - Created 30 cards out of 30 possible lines.


In [29]:
read_package(OUTPUT_DIR / f"{deck_name}.apkg")

Collection(models={5550339091: Model(name='Basic (opt. reversed)', field_names=['Front', 'Back', 'Extra', 'More', 'Phonetics', 'Image', 'Audio'])}, notes=[Note(model=Model(name='Basic (opt. reversed)', field_names=['Front', 'Back', 'Extra', 'More', 'Phonetics', 'Image', 'Audio']), field_contents=['la fenêtre', 'das Fenster', '', '', '/la- fənˈɛtʁ/', '<img src="img_83d2b53e093c284e.png">', ''], tags=set()), Note(model=Model(name='Basic (opt. reversed)', field_names=['Front', 'Back', 'Extra', 'More', 'Phonetics', 'Image', 'Audio']), field_contents=['Monsieur Macron', 'Herr Macron', '(M. Macron)', '', '/məsjˈø makʁˈɔ̃/', '<img src="img_c62070d054b986e3.png">', ''], tags=set()), Note(model=Model(name='Basic (opt. reversed)', field_names=['Front', 'Back', 'Extra', 'More', 'Phonetics', 'Image', 'Audio']), field_contents=['Madame Borne', 'Frau Borne', '(MMe Borne)', '', '/madˈam bˈɔʁn/', '<img src="img_b2f6efc7954ab4d9.png">', ''], tags=set()), Note(model=Model(name='Basic (opt. reversed)', f

In [30]:
def main():
    parser = argparse.ArgumentParser(description='Create Anki decks from CSV or simple Markdown files.')
    parser.add_argument('file', help='File to import (CSV or Markdown). The file must have a header row that defines the fields for the Anki cards.')
    parser.add_argument('--deck-name', default=None, help='Optional: Custom name for the Anki deck. If not provided, the deck name is derived from the file name.')
    parser.add_argument('--package-name', default=None, help='Optional: Custom name for the Anki package file (without .apkg extension). If not provided, the package name is the same as the deck name.')

    args = parser.parse_args()

    file_name = args.file
    deck_name = args.deck_name if args.deck_name else os.path.splitext(os.path.basename(file_name))[0]
    package_name = args.package_name if args.package_name else deck_name + '.apkg'

    if not os.path.exists(file_name):
        print(f"Error: The file {file_name} does not exist.")
        return

    try:
        file_data = read_file(file_name)
    except ValueError as e:
        print(e)
        return
    

    